In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/prep_ab.csv"

In [ ]:
!pip install contractions
!pip install pyLDAvis
!pip install -U gensim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('mode.chained_assignment', None)

import gensim
import re, string
import contractions
from bs4 import BeautifulSoup
from html import unescape

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk.util import ngrams

import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import pyLDAvis.gensim_models
import pyLDAvis

from heapq import nlargest
from collections import Counter
from tqdm import tqdm
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Text normalization functions

In [ ]:
lemmatizer = WordNetLemmatizer()
table = str.maketrans(dict.fromkeys(string.punctuation))

def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+|pic\.twitter\S+|bit.ly/\S+", "", text)

def remove_html_chars(text):
    soup = BeautifulSoup(unescape(text), 'lxml')
    return soup.text

def fix_contractions(text):
    return contractions.fix(text).lower()

def remove_mentions(text):
    return " ".join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text).split())

def remove_hashtags(text):
    return re.sub(r"#(\w+)", " ", text)

def remove_punctuations(text):
    return text.translate(table)

def remove_digits(text):
    return re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", text)

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word, pos='v') for word in text.split()])


Function to preprocess text in the passed column of the dataframe


In [ ]:
def normalize_text(df, col):

    print("Normalizing text and adding column 'processed_text'")
    df['processed_text'] = df[col]

    # Converting to lower case
    df['processed_text'] = df['processed_text'].str.lower()
    # Removing \n characters
    df['processed_text'] = df['processed_text'].apply(lambda text: text.replace('\n', ' '))
    # Removing urls
    df['processed_text'] = df['processed_text'].apply(lambda text: remove_URL(text))
    # Removing HTML chars
    df['processed_text'] = df['processed_text'].apply(lambda text: remove_html_chars(text))
    # Fix contractions
    df['processed_text'] = df['processed_text'].apply(lambda text: fix_contractions(text))
    # Removing the mentions
    df['processed_text'] = df['processed_text'].apply(lambda text: remove_mentions(text))
    # Removing punctuations
    df['processed_text'] = df['processed_text'].apply(lambda text: remove_punctuations(text))
    # Removing the stopwords
    df['processed_text'] = df['processed_text'].apply(lambda text: remove_stopwords(text))
    # Remove the digits
    df['processed_text'] = df['processed_text'].apply(lambda text: remove_digits(text))
    # Lemmatization of text
    df['processed_text'] = df['processed_text'].apply(lambda text: lemmatize_text(text))

    print('DONE! \n')

In [ ]:
def normalize_text_NER(df, col):

    print("Normalizing text for NER and adding column 'processed_text_NER'")
    df['processed_text_NER'] = df[col]

    # Converting to lower case
    df['processed_text_NER'] = df['processed_text_NER'].str.lower()
    # Removing \n characters
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: text.replace('\n', ' '))
    # Removing urls
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: remove_URL(text))
    # Removing HTML chars
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: remove_html_chars(text))
    # Fix contractions
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: fix_contractions(text))
    # Removing the mentions
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: remove_mentions(text))
    # Removing Hashtags
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: remove_hashtags(text))
    # Removing punctuations
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: remove_punctuations(text))
    # Remove the digits
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: remove_digits(text))
    # Lemmatization of text
    df['processed_text_NER'] = df['processed_text_NER'].apply(lambda text: lemmatize_text(text))

    df['processed_text_NER'] = df['processed_text_NER'].str.strip()

    print('DONE! \n')

### Processing tweets

Original / old -

In [ ]:
df = pd.read_csv(path + "Dataset_JUN_14FEB.csv")
df = df[pd.isna(df.text)==False]
df.reset_index(drop=True, inplace=True)
df.tail()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,username,location,tweetcreatedts,text,Country
1084739,DBean48,West Palm,2020-06-01 01:12:22+00:00,@JudicialWatch @realDonaldTrump You bet he was...,USA
1084740,TheSunsRay,"Chestermere, Alberta",2020-06-01 01:08:23+00:00,"Big Pharma is poison, they can vaccinate thems...",USA
1084741,Masc4FemFatale,"New York, NY",2020-06-01 00:51:27+00:00,The way I see people partying and having bbqs ...,USA
1084742,ADNInfoDrive,NaN,2020-06-01 00:40:08+00:00,"Para hindi malito sa ECQ, MECQ, GCQ, MGCQ at k...",India
1084743,bob_wray,Gtr Manchester,2020-06-01 00:39:12+00:00,Someone knows the truth about Covid19 I don’t ...,UK


In [ ]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
# Keep texts greater than 1 word
df = df[df['text'].apply(lambda txt: len(word_tokenize(txt)) > 1)]
df.reset_index(drop=True, inplace=True)

In [ ]:
normalize_text(df, "text")

Normalizing text and adding column 'processed_text'
DONE! 



In [ ]:
normalize_text_NER(df, "text")

Normalizing text for NER and adding column 'processed_text_NER'
DONE! 



In [ ]:
df.tail()

,username,location,tweetcreatedts,text,Country,processed_text,processed_text_NER
986773,DBean48,West Palm,2020-06-01 01:12:22+00:00,@JudicialWatch @realDonaldTrump You bet he was...,USA,bet will doctor care big pharma try squash use...,you bet he be he be will to take under doctor ...
986774,TheSunsRay,"Chestermere, Alberta",2020-06-01 01:08:23+00:00,"Big Pharma is poison, they can vaccinate thems...",USA,big pharma poison vaccinate read mad fear wart...,big pharma be poison they can vaccinate themse...
986775,Masc4FemFatale,"New York, NY",2020-06-01 00:51:27+00:00,The way I see people partying and having bbqs ...,USA,way people party have bbqs make think vaccine ...,the way i see people party and have bbqs make ...
986776,ADNInfoDrive,NaN,2020-06-01 00:40:08+00:00,"Para hindi malito sa ECQ, MECQ, GCQ, MGCQ at k...",India,para hindi malito sa ecq mecq gcq mgcq kung an...,para hindi malito sa ecq mecq gcq mgcq at kung...
986777,bob_wray,Gtr Manchester,2020-06-01 00:39:12+00:00,Someone knows the truth about Covid19 I don’t ...,UK,know truth covid19 know search search vaccine ...,someone know the truth about covid19 i do not ...


#### Update Data / Append new
Replace name of file

In [ ]:
df = pd.read_csv(path + "Brazil-missing-Feb-Mar.csv")
df = df[pd.isna(df.text)==False]
df.reset_index(drop=True, inplace=True)
df.tail()

,username,location,tweetcreatedts,text,Country
132,luisa_braghieri,Capivari SP,2021-03-15 21:42:52+00:00,@BrainAblaze Which vaccine do you have with Ep...,Brazil
133,cgisaopaulo,"Sao Paulo, Brazil",2021-03-15 15:41:22+00:00,#LargestVaccineDrive\n\nMore than 30 million #...,Brazil
134,AlbertoJabur1,NaN,2021-03-15 14:54:00+00:00,"Fauci disse: ""The best way that we can avoid a...",Brazil
135,luisa_braghieri,Capivari SP,2021-03-15 03:53:46+00:00,What vaccine are you getting? my neurologist d...,Brazil
136,Sylvana404,"Piriapolis, Uruguay",2021-03-15 03:40:59+00:00,Benz.. my soon to be adopted puppy... picking ...,Brazil


In [ ]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

# Keep texts greater than 1 word
df = df[df['text'].apply(lambda txt: len(word_tokenize(txt)) > 1)]
df.reset_index(drop=True, inplace=True)

In [ ]:
df.Country.value_counts()

Brazil    137
Name: Country, dtype: int64

In [ ]:
normalize_text(df, "text")

normalize_text_NER(df, "text")

Normalizing text and adding column 'processed_text'
DONE! 

Normalizing text for NER and adding column 'processed_text_NER'
DONE! 



In [ ]:
print(min(df.tweetcreatedts))
print(max(df.tweetcreatedts))

2021-02-15 00:17:00+00:00
2021-03-22 15:58:46+00:00


In [ ]:
old_df = pd.read_csv(path + "tweets_processed.csv")
new_df = old_df.append(df, ignore_index=True)
new_df.drop_duplicates(subset=["username","tweetcreatedts","text"], ignore_index=True, inplace=True)

Save new dataframe

In [ ]:
new_df

,username,location,tweetcreatedts,text,Country,processed_text,processed_text_NER
0,Glen_Maney,London,2021-02-14 23:59:59+00:00,"If you don't agree, don't sign. No need to tel...",UK,agree sign need tell agree feel free sign rt p...,if you do not agree do not sign no need to tel...
1,edinburgh1965,"Scotland, United Kingdom",2021-02-14 23:59:58+00:00,@JuliaHB1 @naomirwolf So why do they “pretend ...,UK,pretend vaccine celebs obvious actually get ja...,so why do they pretend to give this vaccine to...
2,AndrewCrystall,"London, London",2021-02-14 23:59:56+00:00,"@mikekatz Yes, your ignoring your shorting Lon...",UK,yes ignore short london vaccine excuse socalle...,yes your ignore your short london of vaccine a...
3,tmj_mgm_pharm,"Montgomery, AL",2021-02-14 23:59:52+00:00,Want to work at CVS Health? We're hiring in Tr...,USA,want work cvs health hire troy al click link b...,want to work at cvs health we be hire in troy ...
4,MarilenaMelas,"Columbus, OH",2021-02-14 23:59:50+00:00,Love yourself and others on this #ValentinesDa...,USA,love valentinesday lead example hours 2nd dose...,love yourself and others on this valentinesday...
...,...,...,...,...,...,...,...
1865779,luisa_braghieri,Capivari SP,2021-03-15 21:42:52+00:00,@BrainAblaze Which vaccine do you have with Ep...,Brazil,vaccine epilepsy take,which vaccine do you have with epilepsy take i...
1865780,cgisaopaulo,"Sao Paulo, Brazil",2021-03-15 15:41:22+00:00,#LargestVaccineDrive\n\nMore than 30 million #...,Brazil,largestvaccinedrive million covid19 vaccine do...,largestvaccinedrive more than million covid19 ...
1865781,AlbertoJabur1,NaN,2021-03-15 14:54:00+00:00,"Fauci disse: ""The best way that we can avoid a...",Brazil,fauci disse best way avoid threat variants thi...,fauci disse the best way that we can avoid any...
1865782,luisa_braghieri,Capivari SP,2021-03-15 03:53:46+00:00,What vaccine are you getting? my neurologist d...,Brazil,vaccine get neurologist allow refractory epile...,what vaccine be you get my neurologist do not ...


In [ ]:
new_df.to_csv(path + "tweets_processed.csv", index=False)

In [ ]:
df = new_df[new_df.Country.isin(["USA","UK","India","Brazil","Australia"])]
df.reset_index(drop=True,inplace=True)
df.Country.value_counts()
df.to_pickle(path+'empath_df_customized.pkl')

### Save for 5 required countries

## Topic Modeling

### Read processed dataframe

In [ ]:
df = pd.read_csv(path + "tweets_processed.csv")
df

,username,location,tweetcreatedts,text,Country,processed_text
0,julianexcalibur,new york city,2021-01-25 23:59:57+00:00,@FibbyGibby i will be getting the vaccine so i...,USA,get vaccine problem wirh
1,e_preese,"Minnesota, USA",2021-01-25 23:59:57+00:00,The world’s 10 richest billionaires could pay ...,USA,worlds richest billionaires pay world vaccinat...
2,MediaGirlOttawa,"Ottawa, Canada",2021-01-25 23:59:56+00:00,@jonpole Hopefully they didn’t get vaccinated.,USA,hopefully vaccinate
3,JeanneMcNichol4,"Washington, USA",2021-01-25 23:59:56+00:00,@JackPosobiec Went to the doctor today couldn’...,USA,go doctor today vaccine young tho caregiver yr...
4,SteveB231,"Saint Charles, IL",2021-01-25 23:59:56+00:00,It will be very interesting to see where thing...,USA,interest things stand month play vaccine soon
...,...,...,...,...,...,...
842295,DBean48,West Palm,2020-06-01 01:12:22+00:00,@JudicialWatch @realDonaldTrump You bet he was...,USA,bet will doctor care big pharma try squash use...
842296,TheSunsRay,"Chestermere, Alberta",2020-06-01 01:08:23+00:00,"Big Pharma is poison, they can vaccinate thems...",USA,big pharma poison vaccinate read mad fear wart...
842297,Masc4FemFatale,"New York, NY",2020-06-01 00:51:27+00:00,The way I see people partying and having bbqs ...,USA,way people party have bbqs make think vaccine ...
842298,ADNInfoDrive,NaN,2020-06-01 00:40:08+00:00,"Para hindi malito sa ECQ, MECQ, GCQ, MGCQ at k...",India,para hindi malito sa ecq mecq gcq mgcq kung an...


In [ ]:
# Remove columns with NaN processed text
df = df[pd.isna(df.processed_text)==False]
df.reset_index(drop=True, inplace=True)

In [ ]:
df.Country.value_counts()

USA          506088
UK           232624
India         70394
Australia     15686
Brazil         7708
Germany        7038
Italy          2498
Russia          183
Name: Country, dtype: int64

In [ ]:
# Number of unique tokens
all_texts = list(df.processed_text)
items = Counter(" ".join(all_texts).split(" ")).items()
len(items)

238252

### Tokenization


In [ ]:
df = pd.read_csv(path)

In [ ]:
df

,Abstract
0,covid 19 affects multiple vital organs leads d...
1,concomitantly first pandemic wave covid19 unex...
2,workshop two phases phase 1 consisted introduc...
3,dataset part outputs research project virginia...
4,raw data study comparing vitro cytokine oxidat...
...,...
7866,seasonal water table depth measurements collec...
7867,datos de casos en la pandemia de covid 19 en per
7868,survey wp1 household covid 19 ucc ipsos study ...
7869,paper projects aggregate populations six pacif...


In [ ]:
df.dropna(subset = ["Abstract"], inplace=True)

In [ ]:
df.reset_index(inplace = True)

In [ ]:
df["Abstract"][290]

'current covid 19 cases alberta local geographic area mirror data presented map view https www alberta ca stats covid 19 alberta statistics htm geospatial tab vaccination progress data joined https www alberta ca data stats lga coverage csv updated 2021 06 23 16 08 data end day 2021 06 22'

In [ ]:
def get_tokenized_sentences(dataframe, col):
  """
  Input parameters
  dataframe: (pandas dataframe object)
  col      : (str) Name of column containing text
  country  : (str) Name of "Country" to filter

  Output
  (list) tokenized sentences

  """
  print("token")
  df_country = dataframe             #added
  print("t1")
  # if country=="all":
  #   df_country = dataframe
  # else:
  #   df_country =  dataframe[dataframe.Country==country]
  #   df_country.reset_index(drop=True, inplace=True)
  i = 0
  sentences = []
  for Abstract in df_country[col]:
    i += 1
    text = Abstract
    # Removed wide spaces
    text = re.sub(' +', " ", text)
    sentences.append(sent_tokenize(text))
    # print(i,text)

  # print("t3")
  tokenized_sents = []
  for sent in tqdm(sentences):
    # print("t4")
    if len(sent):
      text = sent[0]
      tokens = word_tokenize(text)
      tokenized_sents.append(tokens)

  return tokenized_sents

### Topic Modeling using Gensim

In [ ]:
def get_topic_model(num_topics):

  tokenized_sents = get_tokenized_sentences(df, col="Abstract")
  print("x")
  # Bigram model
  bigram = Phrases(tokenized_sents, min_count=5, threshold=100) # higher threshold fewer phrases.
  bigram_phraser = Phraser(bigram)

  tokenized_sents = [bigram_phraser[sent] for sent in tokenized_sents]

  # Prepare BOW
  dictionary = corpora.Dictionary(tokenized_sents)
  corpus = [dictionary.doc2bow(text) for text in tokenized_sents]

  # Topic Modelling
  ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = num_topics, id2word=dictionary, passes=10, chunksize=10000)

  print("\n")
  pyLDAvis.enable_notebook()
  show(pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary))

  return ldamodel

USA

In [ ]:
key_topics = get_topic_model(num_topics=5)

In [ ]:
usa_topics = get_topic_model(country_name="USA", num_topics=5)

100%|██████████| 506088/506088 [01:05<00:00, 7752.63it/s]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.042508 -0.152628       1        1  28.458963
2      0.079881  0.221350       2        1  19.629514
3     -0.100543  0.134550       3        1  17.844042
0      0.257723 -0.110969       4        1  17.735164
1     -0.194554 -0.092303       5        1  16.332317, topic_info=         Term          Freq         Total Category  logprob  loglift
96    covid19  97587.000000  97587.000000  Default  30.0000  30.0000
284     trump  52776.000000  52776.000000  Default  29.0000  29.0000
114   receive  24227.000000  24227.000000  Default  28.0000  28.0000
182      dose  28454.000000  28454.000000  Default  27.0000  27.0000
19         go  35254.000000  35254.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
300      need   4437.835977  32962.687879   Topic5  -5.4077  -0.1932
218      time   4088.898194  21866.646501   Topic5  -5.4896   0.1353
9    vaccines   3999.055141  21339.827032   Topic5  -5.5118   0.1375
61      covid   4134.423746  59873.704478   Topic5  -5.4785  -0.8609
467      come   3974.409511  23292.147092   Topic5  -5.5180   0.0438

[314 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
115       2  0.798535     1st
115       4  0.201385     1st
179       2  0.999833     2nd
1273      2  0.176722  access
1273      4  0.823074  access
...     ...       ...     ...
144       2  0.069919   years
144       3  0.167954   years
144       4  0.001781   years
144       5  0.084727   years
3505      2  0.999847    york

[436 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 1, 2])

UK

In [ ]:
uk_topics = get_topic_model(country_name="UK", num_topics=3)

100%|██████████| 232624/232624 [00:30<00:00, 7595.83it/s]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.102117  0.158435       1        1  40.643737
2     -0.121307 -0.149615       2        1  30.481124
1      0.223424 -0.008820       3        1  28.875140, topic_info=            Term          Freq         Total Category  logprob  loglift
138    vaccinate  35892.000000  35892.000000  Default  30.0000  30.0000
32   vaccination  34632.000000  34632.000000  Default  29.0000  29.0000
312         dose  15237.000000  15237.000000  Default  28.0000  28.0000
325           uk  17577.000000  17577.000000  Default  27.0000  27.0000
610          nhs   8630.000000   8630.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
138    vaccinate   6370.096089  35892.975865   Topic3  -4.8876  -0.4867
499         week   3334.152693   6690.809505   Topic3  -5.5350   0.5457
24          good   3217.490264   9033.421163   Topic3  -5.5706   0.2099
11        people   3648.118840  32519.894297   Topic3  -5.4450  -0.9455
412         wait   3081.448391   6822.606961   Topic3  -5.6138   0.4473

[207 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
971       2  0.999809      2nd
1599      3  0.999583    amaze
3791      1  0.999171     anti
405       1  0.977083  approve
405       2  0.022932  approve
...     ...       ...      ...
508       2  0.078549     year
508       3  0.522763     year
172       1  0.878021    years
172       2  0.000447    years
172       3  0.121600    years

[291 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

INDIA

In [ ]:
india_topics = get_topic_model(country_name="India", num_topics=5)

100%|██████████| 70394/70394 [00:09<00:00, 7088.39it/s]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.182554 -0.028403       1        1  32.725140
1      0.040496 -0.230524       2        1  23.175642
0     -0.168130  0.031539       3        1  16.295713
2     -0.224576  0.047573       4        1  13.997016
3      0.169657  0.179815       5        1  13.806489, topic_info=             Term          Freq         Total Category  logprob  loglift
111   vaccination  12905.000000  12905.000000  Default  30.0000  30.0000
1         vaccine  51927.000000  51927.000000  Default  29.0000  29.0000
6           india  12640.000000  12640.000000  Default  28.0000  28.0000
38      vaccinate   6104.000000   6104.000000  Default  27.0000  27.0000
13         health   4244.000000   4244.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
1067       trials    385.763062   1288.371384   Topic5  -5.7382   0.7741
344           new    410.843525   1807.339839   Topic5  -5.6752   0.4986
314          take    436.549742   2589.991518   Topic5  -5.6145   0.1995
1012        phase    385.689800   1611.839465   Topic5  -5.7384   0.5499
715        follow    353.411028   1231.419313   Topic5  -5.8258   0.7317

[349 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
304        1  0.019649        1st
304        3  0.979432        1st
43982      4  0.989819  1st_stage
2494       1  0.119710        2nd
2494       3  0.871380        2nd
...      ...       ...        ...
3264       5  0.991386         ye
378        1  0.705663       year
378        2  0.165579       year
378        3  0.061825       year
378        5  0.066089       year

[553 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])

AUSTRALIA

In [ ]:
australia_topics = get_topic_model(country_name="Australia", num_topics=5)

100%|██████████| 15686/15686 [00:02<00:00, 6232.22it/s]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.014035 -0.012074       1        1  29.317008
3     -0.102096 -0.047818       2        1  26.286207
2     -0.096481  0.046633       3        1  18.323157
0      0.114660 -0.110780       4        1  15.163035
1      0.097951  0.124039       5        1  10.910592, topic_info=            Term          Freq         Total Category  logprob  loglift
35     vaccinate   1586.000000   1586.000000  Default  30.0000  30.0000
21       vaccine  12244.000000  12244.000000  Default  29.0000  29.0000
76        pfizer   1223.000000   1223.000000  Default  28.0000  28.0000
36   vaccination   1356.000000   1356.000000  Default  27.0000  27.0000
538          flu    435.000000    435.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
330         wait     60.388529    420.664738   Topic5  -5.8729   0.2744
56          time     66.988486    693.712186   Topic5  -5.7692  -0.1221
40          dose     63.743200    589.140404   Topic5  -5.8188  -0.0084
24     australia     63.872895   1146.386514   Topic5  -5.8168  -0.6720
171          new     57.106279    451.892120   Topic5  -5.9288   0.1469

[421 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3115      2  0.951190   abandon
9205      5  0.948808     abort
1784      1  0.161905  age_care
1784      2  0.012454  age_care
1784      4  0.821980  age_care
...     ...       ...       ...
305       3  0.151394     years
305       4  0.064528     years
305       5  0.079420     years
7566      5  0.912013        yo
1815      1  0.977708       yrs

[877 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 1, 2])

Brazil

In [ ]:
brazil_topics = get_topic_model(country_name="Brazil", num_topics=4)

100%|██████████| 7708/7708 [00:00<00:00, 7997.11it/s]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.015071  0.012885       1        1  34.106574
1      0.071629  0.053200       2        1  27.676889
2      0.009831 -0.097629       3        1  21.711097
0     -0.096531  0.031544       4        1  16.505441, topic_info=          Term        Freq       Total Category  logprob  loglift
144  vaccinate  968.000000  968.000000  Default  30.0000  30.0000
4         dose  586.000000  586.000000  Default  29.0000  29.0000
482     brazil  446.000000  446.000000  Default  28.0000  28.0000
289        say  458.000000  458.000000  Default  27.0000  27.0000
708       plan  292.000000  292.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
289        say   52.053155  458.504202   Topic4  -5.6340  -0.3742
224       help   40.421502  200.101018   Topic4  -5.8869   0.2020
76       covid   53.922208  687.092977   Topic4  -5.5987  -0.7434
437       want   43.605284  282.814617   Topic4  -5.8111  -0.0681
122         go   40.993447  288.307607   Topic4  -5.8728  -0.1491

[319 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1109      1  0.930568              1b
495       2  0.942802       1st_shoot
1302      2  0.981282    60plus_weeks
171       1  0.031773      administer
171       2  0.942598      administer
...     ...       ...             ...
4774      4  0.904126  worlds_largest
133       1  0.166439            year
133       2  0.242093            year
133       3  0.431229            year
133       4  0.158874            year

[559 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1])

Germany

In [ ]:
germany_topics = get_topic_model(country_name="Germany", num_topics=3)

100%|██████████| 7038/7038 [00:00<00:00, 7590.16it/s]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.051100  0.060591       1        1  42.961853
1     -0.041566 -0.064894       2        1  31.788626
2      0.092666  0.004303       3        1  25.249521, topic_info=          Term         Freq        Total Category  logprob  loglift
11   vaccinate  1146.000000  1146.000000  Default  30.0000  30.0000
68      people   909.000000   909.000000  Default  29.0000  29.0000
339     pfizer   468.000000   468.000000  Default  28.0000  28.0000
267       dose   539.000000   539.000000  Default  27.0000  27.0000
19     moderna   162.000000   162.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
213         eu    78.333404   372.925771   Topic3  -5.6672  -0.1840
11   vaccinate   106.934222  1146.265975   Topic3  -5.3560  -0.9957
151       need    65.179721   350.325968   Topic3  -5.8510  -0.3054
68      people    72.315183   909.294830   Topic3  -5.7471  -1.1553
279      think    61.781459   331.885845   Topic3  -5.9046  -0.3048

[278 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
1435      1  0.095603              2021
1435      2  0.095603              2021
1435      3  0.860424              2021
3837      2  0.887602             admin
5791      2  0.933428  adult_population
...     ...       ...               ...
601       1  0.686591              year
601       2  0.208058              year
601       3  0.097094              year
140       1  0.155489           younger
140       2  0.829275           younger

[464 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

Italy

In [ ]:
italy_topics = get_topic_model(country_name="Italy", num_topics=5)

100%|██████████| 2498/2498 [00:00<00:00, 7393.39it/s]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.020577  0.102075       1        1  36.671623
3     -0.128105 -0.029813       2        1  20.288385
0      0.043462 -0.020863       3        1  18.155166
1      0.000696  0.028013       4        1  13.900480
4      0.063369 -0.079413       5        1  10.984347, topic_info=           Term        Freq       Total Category  logprob  loglift
371   care_home   74.000000   74.000000  Default  30.0000  30.0000
1082    england   74.000000   74.000000  Default  29.0000  29.0000
377    scotland   62.000000   62.000000  Default  28.0000  28.0000
53    vaccinate  328.000000  328.000000  Default  27.0000  27.0000
24         dose  208.000000  208.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
835        help    7.373406   38.861017   Topic5  -6.1278   0.5466
189      pfizer   11.232306  205.213110   Topic5  -5.7069  -0.6966
16        italy    8.919768   93.913049   Topic5  -5.9374  -0.1454
45         like    8.667687   94.973909   Topic5  -5.9661  -0.1853
38         come    7.250407   55.702906   Topic5  -6.1446   0.1697

[390 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
2541      4  0.217590                   200k
2541      5  0.652769                   200k
521       1  0.921963                   2021
521       2  0.076830                   2021
3031      5  0.652457              academics
...     ...       ...                    ...
2816      5  0.652308                younger
4015      3  0.871579  zonaarancione_vaccino
4015      4  0.124511  zonaarancione_vaccino
956       3  0.186029              zonarossa
956       5  0.744117              zonarossa

[745 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2, 5])